### This  file is specific to preprocessing and first-round data cleaning requirements. Take the new JSON files generated from here and use it separately for the purposes of exploration and modelling

In [ ]:
import numpy as np
import scipy
import random
import urllib
import matplotlib.pyplot as plt
import json

In [ ]:
random.seed(0) #set an initial seed so that we can reproduce consistently

In [ ]:
def readJSONfromFile(fname):
    yelp_data = []
    with open(fname) as f:
        for line in f:
            yelp_data.append(json.loads(line))
    return yelp_data

In [ ]:
Iu = readJSONfromFile('data/yelp_Iu.json')

In [ ]:
Iu = Iu[0]

In [ ]:
len(Iu)

In [ ]:
Iu['PmgqNO0-5Y3e3UoR61TD7w']

In [ ]:
I = readJSONfromFile('data/yelp_restaurant_data.json')

In [ ]:
I = I[0]

In [ ]:
AllRest = []
for i in I:
    AllRest.append(i['business_id'])

In [ ]:
AllUser = Iu.keys()

In [ ]:
len(AllUser)

In [ ]:
len(AllRest)

In [ ]:
AllRestSet = set(AllRest)

In [ ]:
Lambda = 3
Alpha = 10**-5
K = 100

In [ ]:
Gamma_u = {}
Gamma_r = {}

In [ ]:
for user in AllUser:
    Gamma_u[user] = np.random.rand(K)
for rest in AllRest:
    Gamma_r[rest] = np.random.rand(K)

In [ ]:
def logit(t):
    return np.exp(-t)/(1+np.exp(-t))

In [ ]:
def sigmoid(t):
    return 1-logit(t)

In [ ]:
import math
def vlen(t):
    su = 0
    for i in t:
        su += i*i
    return su

In [ ]:
CUser = []
Cirest = []
Cjrest = []
iterations = 1000
l=[]

In [ ]:
def loss(CUser,Cirest,Cjrest):
    su = 0
    for i in range(len(CUser)):
        user = CUser[i]
        irest = Cirest[i]
        jrest = Cjrest[i]
        xuij = np.dot(Gamma_u[user],Gamma_r[irest]-Gamma_r[jrest])
        su += np.log(sigmoid(xuij))
    for user in CUser:
        su -= Lambda* vlen(Gamma_u[user])
    for rest in Cirest:
        su -= Lambda* vlen(Gamma_r[rest])
    for rest in Cjrest:
        su -= Lambda* vlen(Gamma_r[rest])
    return su*1.0/len(CUser)

In [ ]:
# for i in range(iterations):
#     user = random.choice(Iu.keys())
#     CUser.append(user)
#     irest = random.choice(Iu[user])
#     Cirest.append(irest)
#     jrest = random.choice(list(AllRestSet - set(Iu[user])))
#     Cjrest.append(jrest)
# for i in range(iterations):
#     user = CUser[i]
#     irest = Cirest[i]
#     jrest = Cjrest[i]
#     xuij = np.dot(Gamma_u[user],Gamma_r[irest]-Gamma_r[jrest])
#     Gamma_u[user] += Alpha*(logit(xuij)*(Gamma_r[irest]-Gamma_r[jrest]) + Lambda*Gamma_u[user])
#     Gamma_r[irest]+= Alpha*(logit(xuij)*(Gamma_u[user]) + Lambda*Gamma_r[irest])
#     Gamma_r[jrest]+= Alpha*(logit(xuij)*(-Gamma_u[user]) + Lambda*Gamma_r[jrest])
#     l.append(loss(CUser,Cirest,Cjrest))

In [ ]:
# plt.plot(range(iterations),l)
# plt.show()

In [ ]:
for i in range(iterations):
    user = random.choice(Iu.keys())
    CUser.append(user)
    irest = random.choice(Iu[user])
    Cirest.append(irest)
    jrest = random.choice(list(AllRestSet - set(Iu[user])))
    Cjrest.append(jrest)
    xuij = np.dot(Gamma_u[user],Gamma_r[irest]-Gamma_r[jrest])
#     x = Alpha*(logit(xuij)*(Gamma_r[irest]-Gamma_r[jrest]) + Lambda*Gamma_u[user])
#     y = Alpha*(logit(xuij)*(Gamma_u[user]) + Lambda*Gamma_r[irest])
#     z = Alpha*(logit(xuij)*(-Gamma_u[user]) + Lambda*Gamma_r[jrest])
    x= Alpha*(logit(xuij)*(Gamma_r[irest]-Gamma_r[jrest]))
    y = Alpha*(logit(xuij)*(Gamma_u[user]))
    z = Alpha*(logit(xuij)*(-Gamma_u[user]))
    for user in CUser:
        x += Alpha*Lambda*Gamma_u[user]
    for rest in Cirest:
        y += Alpha*Lambda*Gamma_r[irest]
    for rest in Cjrest:
        z += Alpha*Lambda*Gamma_r[jrest]
    Gamma_u[user] += x
    Gamma_r[irest] += y
    Gamma_r[jrest] += z
    l.append(loss(CUser,Cirest,Cjrest))

In [ ]:
plt.plot(range(iterations),l)
plt.show()